In [24]:
import numpy as np



$ h(x) = \sum_{i=1}^{N} \frac{1}{8^i} {(x_i-2^i)}^2$

For N = 3 

$ h(x) = \frac{1}{8} {(x_1-2)}^2 + \frac{1}{64} {(x_2-4)}^2 + \frac{1}{512} {(x_3-8)}^2$


$ \nabla h(x) = (\frac{1}{4} {(x_1-2)} , \frac{1}{32} {(x_2-4)},\frac{1}{256} {(x_3-8)}  )$

A (d x d)  = $diag(1/8,1/8^2,1/8^3,...,1/8^d)$ it is a diagonal  matrix  in this problem

In [8]:
def evalf(x,d):
  assert type(x) is np.ndarray and len(x) == d
  list1 = [ 1/(8**(i))*((x[i-1]-(2**i))**2) for i in range(1,d+1)]
  return sum(list1)

In [9]:
def evalg(x,d):
  assert type(x) is np.ndarray and len(x) == d
  return np.array([2 * 1/(8**(i)) *(x[i-1]-(2**i)) for i in range(1,d+1)])

In [10]:
def compute_steplength_exact(gradf,A,d):
  assert type(gradf) is np.ndarray and len(gradf) == d
  assert type(A) is np.ndarray and A.shape[0] == d and A.shape[1] == d

  a = np.dot(gradf.T,gradf)
  b = 2* np.dot(np.dot(gradf.T,A),gradf)
  step_length = a/b
  return step_length

In [11]:
def compute_steplength_backtracking(x,gradf,alpha_start,rho,gamma,d):
  assert type(x) is np.ndarray and len(x) == d
  assert type(gradf) is np.ndarray and len(gradf) == d

  alpha = alpha_start
  while evalf(x-alpha*gradf,d) > (evalf(x,d) - gamma * alpha * np.dot(gradf.T,gradf)):
    alpha = rho * alpha
  return alpha

In [12]:
EXACT_LINE_SEARCH = 1
BACKTRACKING_LINE_SEARCH = 2
CONSTANT_STEP_LENGTH = 3

In [21]:
def find_minimizer(d,start_x, tol, line_search_type, *args):
  assert type(start_x) is np.ndarray and len(start_x) == d
  assert type(tol) is float and tol>=0  
  A = np.diag([(1/8)**i for i in range(1,d+1)])
  x = start_x
  g_x = evalg(x,d)

  #initialization for backtracking line search
  if(line_search_type == BACKTRACKING_LINE_SEARCH):
    alpha_start = args[0]
    rho = args[1]
    gamma = args[2]
    print('Params for Backtracking LS: alpha start:', alpha_start, 'rho:', rho,' gamma:', gamma)

  k = 0
  #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol): 
    if line_search_type == EXACT_LINE_SEARCH:
      step_length = compute_steplength_exact(g_x, A,d) 
    elif line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking(x,g_x, alpha_start,rho, gamma,d)
    elif line_search_type == CONSTANT_STEP_LENGTH: #do a gradient descent with constant step length
      step_length = 0.1
    else:  
      raise ValueError('Line search type unknown. Please check!')
  
    x = np.subtract(x, np.multiply(step_length,g_x)) #update x = x - step_length*g_x
    k += 1
    g_x = evalg(x,d) 

    #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  return x,k


2. [R] Note down the minimizer and minimum function value of h(x).

Solution : Minimizer (2,4,8) and Mininum function value = 0 at minimizer

For Exact line search
* The minimizer: [2.         4.         7.99999998]
* Function value: 9.150071377581033e-19
* No of iterations :  269

Params for Backtracking LS: alpha start: 1 rho: 0.5  gamma: 0.5
For Backtracking line search
* The minimizer: [2.         4.         7.99999997]
* Function value: 1.2748574165464873e-18
* No of iterations :  4964



3. [R] Consider stopping tolerance $τ = 10^{-10}$ and starting point $x_0 = ( 1/
64 , 1/
8 , 1)$. Compare the number of iterations
taken by the gradient descent procedure which uses exact step length computation against the gradient descent
procedure which uses the backtracking line search procedure (with $α_0$ = 1, ρ = 0.5, γ = 0.5). Comment on your
observations.


sol. 

For Constant step length
* The minimizer: [2.         4.         7.99999997]
* Function value: 1.2793410906926164e-18
* No of iterations :  49723

For Exact line search
* The minimizer: [2.         4.         7.99999998]
* Function value: 9.150071377581033e-19
* No of iterations :  269

Params for Backtracking LS: alpha start: 1 rho: 0.5  gamma: 0.5
For Backtracking line search
* The minimizer: [2.         4.         7.99999997]
* Function value: 1.2748574165464873e-18
* No of iterations :  4964

Observation : Number of iterations using exact line search is less than constant step length and backtracking line search and they are in the order exact line search < backtracking line search < constant step length 

In [19]:
my_start_x = np.array([1/64,1/8,1])
my_tol= 1e-10
d = 3

x_opt,k = find_minimizer(3,my_start_x, my_tol, CONSTANT_STEP_LENGTH)
print("For Constant step length")
print("* The minimizer:",x_opt)
print("* Function value:",evalf(x_opt,d))
print("* No of iterations : ",k)

x_opt_els,k = find_minimizer(3,my_start_x, my_tol, EXACT_LINE_SEARCH)
print("For Exact line search")
print("* The minimizer:",x_opt_els)
print("* Function value:",evalf(x_opt_els,d))
print("* No of iterations : ",k)
#check what happens when you call find_minimzer using backtracking line search
x_opt_bls,k = find_minimizer(3,my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, 1, 0.5,0.5)
print(" For Backtracking line search")
print("* The minimizer:",x_opt_bls)
print("* Function value:",evalf(x_opt_bls,d))
print("* No of iterations : ",k)

For Constant step length
* The minimizer: [2.         4.         7.99999997]
* Function value: 1.2793410906926164e-18
* No of iterations :  49723
For Exact line search
* The minimizer: [2.         4.         7.99999998]
* Function value: 9.150071377581033e-19
* No of iterations :  269
Params for Backtracking LS: alpha start: 1 rho: 0.5  gamma: 0.5
 For Backtracking line search
* The minimizer: [2.         4.         7.99999997]
* Function value: 1.2748574165464873e-18
* No of iterations :  4964


4. [R] Check if similar observations hold in the N = 4 case as well. Choose a starting point x0 = ( 1/
512 , 1/
64 , 1/
8 , 1)
and for the backtracking line search procedure, use $α_0$ = 1, ρ = 0.5, γ = 0.5. Comment on your observations.

sol :

For Constant step length
* The minimizer: [ 2.         4.         8.        15.9999998]
* Function value: 1.0239049236661478e-17
* No of iterations :  370870

For Exact line search
* The minimizer: [ 2.          4.          8.         15.99999981]
* Function value: 8.8565993523583e-18
* No of iterations :  2013

Params for Backtracking LS: alpha start: 1 rho: 0.5  gamma: 0.5
For Backtracking line search
* The minimizer: [ 2.         4.         8.        15.9999998]
* Function value: 1.0237544252113035e-17
* No of iterations :  37079


Observation : Number of iterations using exact line search is less than constant step length and backtracking line search and they are in the order exact line search < backtracking line search < constant step length 

In [23]:
my_start_x = np.array([1/512,1/64,1/8,1])
my_tol= 1e-10
d = 4

x_opt,k = find_minimizer(4,my_start_x, my_tol, CONSTANT_STEP_LENGTH)
print("For Constant step length")
print("* The minimizer:",x_opt)
print("* Function value:",evalf(x_opt,d))
print("* No of iterations : ",k)

x_opt_els,k = find_minimizer(4,my_start_x, my_tol, EXACT_LINE_SEARCH)
print("For Exact line search")
print("* The minimizer:",x_opt_els)
print("* Function value:",evalf(x_opt_els,d))
print("* No of iterations : ",k)
#check what happens when you call find_minimzer using backtracking line search
x_opt_bls,k = find_minimizer(4,my_start_x, my_tol, BACKTRACKING_LINE_SEARCH, 1, 0.5,0.5)
print(" For Backtracking line search")
print("* The minimizer:",x_opt_bls)
print("* Function value:",evalf(x_opt_bls,d))
print("* No of iterations : ",k)

For Constant step length
* The minimizer: [ 2.         4.         8.        15.9999998]
* Function value: 1.0239049236661478e-17
* No of iterations :  370870
For Exact line search
* The minimizer: [ 2.          4.          8.         15.99999981]
* Function value: 8.8565993523583e-18
* No of iterations :  2013
Params for Backtracking LS: alpha start: 1 rho: 0.5  gamma: 0.5
 For Backtracking line search
* The minimizer: [ 2.         4.         8.        15.9999998]
* Function value: 1.0237544252113035e-17
* No of iterations :  37079


5. [R] Can you also comment on the possible observations for N > 4 case as well, without actually running the
program?


Solution : From the results above for N = 3 to N = 4 we can observe the no of iteration in every method was increased by a significant amount. So we can see that for N > 4 the number of iteration for each method keeps on increasing and the order of number of iterations will be same for every N exact line search < backtracking line search < constant step length
